# Iterative Prisoner's Dilemma


### Description

The [Prisoner's Dilemma](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma) (PD) is a classical game analyzed in game theory, which is widely used to (attempt to) model social/economical interaction. It's a "dilemma" as, if exploited to explain the emergence of altruism in human or in general animal society, it fails badly at a first glance.

The classical situation-representation of the PD is that of two prisoners whose conviction depends on their mutual cooperation. It is easier understood though if illustrated in terms of a trade-off game (closed bag exachange):

*Two people meet and exchange closed bags, with the understanding that one of them contains money, and the other contains a purchase. Either player can choose to honor the deal by putting into his or her bag what he or she agreed, or he or she can defect by handing over an empty bag.*

It is obvious that for both players the winning strategy is to NOT cooperate.

Things changes when the interaction between the two individuals is iterated, in that case a more altruist attitude (strategy) is expected to emerge. The goal of this project is to test this hypothesis.

Mathematically the PD can be expressed with very basic linear algebra. The key component is the **Payoff matrix** $M$, which quantify the reward each player gets depending on whether she cooperated or not (defect):

$$
M = 
\begin{pmatrix} 
R & S \\
T & P 
\end{pmatrix}
$$

with $T,R,S,P$ integers that satisfy the following conditions:

$$
T>R>P>S; \quad 2R > T+S
$$

for example $T=3$, $R=2$, $P=1$ and $S=0$, or  $T=5$, $R=3$, $P=2$, $S=0$. Each player choice (move) can be represented by one of the two axis in ${\rm I\!R}^2$, i.e. $u_C=\begin{pmatrix} 1 \\ 0 \end{pmatrix}$ or $u_D=\begin{pmatrix} 0 \\ 1 \end{pmatrix}$, where the first coordinate stands for *Cooperate* and the second for *Defect*. Being $u_1$ and $u_2$ their rewards $r_1$ and $r_2$ can be computed then as:

$$
r_1 = u_1^T M u_2
\quad
\quad
r_2 = u_2^T M u_1
$$

In an Iterative Prisoner's Dilemma (IPD), two players play prisoner's dilemma more than once in succession and they remember previous actions of their opponent and change their strategy accordingly. The winning strategy is the one which yields to a larger reward at the end of the IPD.

The strategy can be represented as a function which outputs either $u_C$ or $u_D$. Such function can depend on the opponent's history of moves, her on history of moves, on the number of moves played till that moment and so on, but it can only be based on a probability density function. Possible strategies are:

* **Nice guy**: always cooperate (the function's output is always $u_D$)
* **Bad guy**: always defect 
* **Mainly nice**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k<50$
* **Mainly bad**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k>50$
* **tit-for-tat**: start by cooperating, then repeat what the opponent has done in the previous move 

Many more and much more complex strategies can be implemented. The strategy can even change during the IPD.


### Assignments

* Implement a simple IPD between two players implementing two given strategies. Study the evolution along the tournament confronting different strategies; study the overall outcome in the different configurations. 
* Implement a multiple players IPD (MPIPD) where several strategies play against each other in a roud-robin scheme
* Iterate what done in the previous task (repeated MPIPD, rMPIPD)  by increasing the population implementing a given strategy depending on the results that strategy achieved in the previous iteration
* (*difficult*) Implement a rMPIPD where strategies are allowed to mutate. The goal is to simulate the effect of genetic mutations and the effect of natura selection. A parameter (gene) should encode the attidue of an individual to cooperate, such gene can mutate randomly and the corresponding phenotype should compete in the MPIPD such that the best-fitted is determined.  


In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
from functools import partial
import pandas as pd
from IPython.display import display

%precision 1

'%.1f'

In [2]:
uc, ud = [1,0], [0,1]
k=0.3

def nice_guy(it,u):
    return uc
def bad_guy(it,u):
    return ud
def mainly_nice(it,u):
    a = npr.rand()
    if a > k: return uc
    else: return ud
def mainly_bad(it,u):
    a = npr.rand()
    if a <= k: return uc
    else: return ud
def tit_tat(it,u):
    if it==1: return uc
    else: return u  

strat = {'nice': partial(nice_guy),
        'bad': partial(bad_guy), 
        'm_nice': partial(mainly_nice),
        'm_bad': partial(mainly_bad),
        'tit_tat': partial(tit_tat)}


In [3]:
def fight(f,g,N=None,graph=False):

    if N == None: N = 100
            
    R, S, T, P = 3, 0, 5, 1
    M = np.array([[R,S],[T,P]])

    p1, p2 = [-1], [-1]
    for i in range(1,N+1):
        p1.append(strat[f](i,p2[i-1]))
        p2.append(strat[g](i,p1[i-1]))

    p1 = np.array(p1[1:]).T
    p2 = np.array(p2[1:]).T

    result_1 = np.cumsum([np.dot(p1[:,i].T,np.dot(M,p2[:,i])) for i in range(N)])
    result_2 = np.cumsum([np.dot(p2[:,i].T,np.dot(M,p1[:,i])) for i in range(N)])

    if graph == True:
        plt.xlabel('iteration')
        plt.ylabel('points')
        plt.plot(result_1, label=f)
        plt.plot(result_2, label=g)
        plt.legend()

    return [result_1[-1], result_2[-1]]

In [14]:
s = ['nice','bad','m_nice','m_bad','tit_tat']

result = np.zeros((5,5))
for i in range(len(s)):
    for j in range(i,len(s)):
        p1, p2 = fight(s[i],s[j])
        result[i,j] = p1
        result[j,i] = p2

df = pd.DataFrame(result,dtype=int,columns=s,index=s)

def colorcell(x):
    data = x.copy()
    data.loc[:,:] = ''
    for i in range(5):
        for j in range(5):
            if result[i,j] < result[j,i]: data.iloc[i,j] = 'color : red'
            elif result[i,j] > result[j,i]: data.iloc[i,j] = 'color : green'
            else: data.iloc[i,j] = 'color : lightblue'
    return data

sum = np.sum(result, axis=1)
df['result'] = sum
df = df.astype({'result': int})
df = df.style.apply(colorcell,axis=None)

display(df)

,nice,bad,m_nice,m_bad,tit_tat,result
nice,300,0,204,108,300,912
bad,500,100,384,208,104,1296
m_nice,364,29,270,114,248,1025
m_bad,428,73,319,162,186,1168
tit_tat,300,99,248,186,300,1133


In [15]:
#Point 2

N = 20
h = npr.randint(0,5,size=N)
partecipants = [s[i] for i in h]

result = np.zeros((N,N))
for i in range(N):
    for j in range(i):
        p1, p2 = fight(partecipants[i],partecipants[j])
        result[i,j] = p1
        result[j,i] = p2




[[  0. 256. 300. 259.  99. 300. 300. 300.  99.  99. 300.  99. 300. 261.
  269.  99. 174. 300. 264. 255.]
 [256.   0. 362. 242.  21. 362. 254. 352.  27.  30. 352.  25. 354. 257.
  282.  22. 120. 362. 258. 277.]
 [300. 207.   0. 213.   0. 300. 300. 300.   0.   0. 300.   0. 300. 216.
  201.   0. 117. 300. 210. 213.]
 [259. 272. 358.   0.  28. 344. 260. 356.  28.  31. 362.  29. 344. 253.
  282.  35. 118. 348. 269. 249.]
 [104. 416. 500. 388.   0. 500. 104. 500. 100. 100. 500. 100. 500. 368.
  380. 100. 260. 500. 384. 420.]
 [300. 207. 300. 234.   0.   0. 300. 300.   0.   0. 300.   0. 300. 213.
  213.   0.  93. 300. 219. 207.]
 [300. 254. 300. 260.  99. 300.   0. 300.  99.  99. 300.  99. 300. 267.
  251.  99. 178. 300. 264. 257.]
 [300. 222. 300. 216.   0. 300. 300.   0.   0.   0. 300.   0. 300. 207.
  225.   0.  78. 300. 216. 216.]
 [104. 392. 500. 388. 100. 500. 104. 500.   0. 100. 500. 100. 500. 384.
  408. 100. 208. 500. 384. 376.]
 [104. 380. 500. 376. 100. 500. 104. 500. 100.   0. 500